In [250]:
import json
import os
from openai import OpenAI               
import pandas as pd
from tqdm import tqdm
import random                  
import time         
from typing import Any, Dict, List
from dotenv import load_dotenv
from gigachat import GigaChat
import re
load_dotenv()

True

In [184]:
# Пути
INPUT_FILE  = "telegram_posts.json"     # откуда берём посты
OUTPUT_FILE = "data/posts/generated_query_post_pairs.json"

# LM Studio сервер
API_BASE    = "http://localhost:1234/v1"          
API_KEY     = "lm-studio"                         # обычно любой строкой, можно оставить
# MODEL_NAME  = "Qwen/Qwen3-30B-A3B-Instruct-2507"
# MODEL_NAME  = "meta-llama-3.1-8b-instruct"
MODEL_NAME  = "glm-4-32b-0414"





# Параметры генерации
NUM_DESCRIPTIONS = 3
# TEMPERATURE      = 0.75
TEMPERATURE      = 0.8
MAX_TOKENS       = 800
TOP_P            = 0.92
MIN_DESCRIPTION_SYMBOLS = 1500
MAX_DESCRIPTION_SYMBOLS = 2000


def random_scores() -> tuple[float, float, float]:
    """
    Возвращает кортеж из трёх случайных чисел в диапазонах:
      - первый:  0.00 – 0.25
      - второй:  0.30 – 0.70
      - третий:  0.75 – 1.00
    """
    low    = round(random.uniform(0.00, 0.25), 2)
    medium = round(random.uniform(0.30, 0.70), 2)
    high   = round(random.uniform(0.8, 1.00), 2)
    
    return (low, medium, high)

SCORES = random_scores()

print('SCORES:', SCORES)

# Уровни релевантности (можно менять количество, названия, scores)
RELEVANCE_LEVELS = [
    {"label": "совершенно нерелевантно",   "score": SCORES[0]},
    # {"label": "слабо релевантно",          "score": 0.35},
    {"label": "средне релевантно",         "score": SCORES[1]},
    {"label": "очень релевантно / идеально", "score": SCORES[2]},
]





BATCH_SIZE  = 10                        # сколько постов обрабатывать за один запуск (для отладки)
SAVE_EVERY  = 20                        # сохранять промежуточный результат каждые N постов

SCORES: (0.1, 0.56, 0.93)


In [239]:
PROMPT_TEMPLATE = """\
Ты — генератор синтетических описаний товаров строго по заданной степени релевантности.  
Твоя задача — создать описания, которые **точно соответствуют указанному score**, а не просто "какие-то похожие".

На основе поста из Telegram:
«{post_text}»

Сгенерируй РОВНО {num_desc} описаний товаров. Каждое описание должно иметь **строго заданную** степень релевантности:

{relevance_instructions}

Важные правила строгого соблюдения score:
• score = {low_score} → описание **полностью не связано** с постом. Товар должен быть из другой тематики, без единой общей темы, слова или ассоциации. Никаких намёков на содержание поста!
• score = {mid_score} → описание имеет **очень слабую, косвенную связь** (например, общая сфера, но не по сути поста). Связь должна быть минимальной, почти незаметной. Большинство людей сказали бы: "почти не связано".
• score = {high_score} → описание **идеально подходит** к основной идее поста, решает проблему или напрямую связано с ключевыми словами/примерами из текста.

Чтобы заставить тебя строго следовать score:
1. Сначала подумай шаг за шагом: какая главная тема поста? Какие слова/идеи нельзя использовать в низко-релевантных описаниях?
2. Для score ≤ {mid_score} **запрещено** использовать любые ключевые слова, понятия или близкие ассоциации из поста.
3. Описания должны быть **заметно разными** по тематике и стилю.
4. Если описание кажется слишком близким к более высокому score — понизь релевантность намеренно.

ЖЁСТКИЕ ОГРАНИЧЕНИЯ (обязательны к выполнению):
• Длина каждого описания — от {min_description_symbols} до {max_description_symbols}
  символов включительно. Если длина выходит за пределы диапазона, описание считается НЕВАЛИДНЫМ
  и должно быть переписано.

• Описание должно выглядеть как реалистичное карточное описание товара для маркетплейса
• Естественный, продающий русский язык
• СТРОГО ЗАПРЕЩЕНО использовать:
  те же товары, предметы или услуги, которые прямо или косвенно упомянуты в исходном посте;
  синонимы, переформулировки или частные варианты этих товаров.
• Для описаний со score > 0.5:
  товар обязан быть тематически близким к теме поста,
  но НЕ МОЖЕТ быть тем же самым товаром или его разновидностью.

Перед финальным ответом проверь каждое описание:
    длина в символах,
    отсутствие совпадений с товарами из поста,
    соответствие формату маркетплейса.

Если хотя бы одно условие нарушено — исправь описание до соответствия.

Примеры РЕАЛЬНЫХ описаний товаров. Изучи их структуру, стиль, как они написаны. При генерации ориентируйся на эти примеры.

• Термометр комнатный:
  Температура и влажность в помещении непосредственно влияют на наше здоровье, 
  резкие перепады могут привести к сухости в горле, жару, 
  аллергии и просто неприятным ощущениям. Датчик температуры и влажности Xiaomi 
  второго поколения создан специально для чуткого наблюдения за изменением микроклимата, 
  а для еще лучшего контроля датчик можно включить в умный сценарий, создавая различные 
  сочетания с другими устройствами. При помощи Bluetooth-шлюза датчик можно использовать 
  в умных сценариях для совместной работы с другими устройствами, чтобы взять микроклимат 
  в помещении под полный контроль и вручную контролировать температуру и влажность для обеспечения самых комфортных показателей. В приложении Mi Home можно настроить сценарий, 
  при котором по достижении определенной температуры автоматически включается или выключается 
  кондиционер, обогреватель или увлажнитель. Особый детский режим работы датчика умеет автоматически 
  присылать уведомления, если температура или влажность в помещении выходят за комфортные 
  для ребенка пределы. На этот режим также можно настроить работу других устройств, 
  позаботившись о здоровье и комфорте всех членов семьи. Используя простые и понятные смайлики, 
  датчик поможет понять состояние микроклимата, чтобы Вы могли позаботиться о самых близких.

• Зеркало для макияжа, зеркало с подсветкой для макияжа настольное косметическое:
  Этот стильный предмет не только красиво дополнит интерьер вашего ванны или спальни, 
  но и станет незаменимым помощником для создания идеального макияжа. 
  Зеркало с увеличением, предлагает вам возможность видеть каждую деталь вашего образа 
  с удивительной четкостью. Круглое зеркало с подсветкой для макияжа с увеличением, создаст идеальное освещение,
  позволяющее точно наносить макияж и ухаживать за кожей.
  Комплектация: 
  - зеркало 1 шт;
  - мини зеркало 1 шт;
  - провод usb 1 шт.

• Легинсы Lo-Lo Для беременных:
  Для будущей мамы важно, чтобы одежда была удобной, легко подстраивалась под быстро меняющееся 
  во время беременности тело, мягко охватывала животик и не сковывала движений. 
  И при этом женщине, готовящейся стать мамочкой, тоже хочется выглядеть современно и стильно. 
  Классические однотонные лосины прекрасно сочетают в себе все эти достоинства. 
  Наши леггинсы предназначены дарить чувство комфорта и свободы. Повседневные лосины выполнены 
  из очень эластичного трикотажа. Тонкие лосины бережно облегают ноги и живот. 
  Правильно подобранные по размеру леггинсы не просвечивают. Особенный крой лосин обеспечит 
  идеальную посадку по фигуре. Широкая резинка в поясе отлично тянется и не давит на растущий 
  животик даже на поздних сроках. Базовые лосины можно носить на ранних сроках и в последнем 
  триместре до дня родов. И после рождения малыша эти узкие штаны все также будут незаменимой вещью 
  в гардеробе.

  
• Себозол противогрибковый шампунь от перхоти, против псориаза, себореи, корочек, лишая с кетоконазолом 100мл:
Шампунь для наружного применения в дерматологии и косметологии. Оказывая противомикробное и противогрибковое действие, устраняет не только симптомы, но и причину появления перхоти.
Шампунь "Себозол" разработан специально для удаления перхоти (воздействует на грибковые 
поражения кожи головы, лица и туловища). 
Его можно использовать при заболеваниях, 
сопровождающихся следующими проявлениями: перхоть, себорейный дерматит 
(красно-коричневые бляшки с шелушением), отрубевидный лишай, себорейный псориаз.
  
• Ортодонтические резинки для брекетов / Тяги для брекетов (эластики) - Медведь / Bear (6,35 мм., 130 гр.) Ormco:

Ортодонтические эластики для брекетов Медведь / Bear из серии Zoo ("Зоопарк")
 произведены одним из самых известных и надежных брендов - Ormco. 
 Резинки Ормко отличаются качеством и долговечностью, что делает их популярным выбором среди ортодонтов
   и пациентов по всему миру. По градации разработанной Ormco, тяги для брекетов Медведь относятся 
   к сильным, их тяговое усилие составляет 4,5 oz / 130 гр, а диаметр кольца равен 1/4'' - 6.35 мм. 
   Важно помнить, заявленное усиление тяги для брекетов достигают при растяжении их в 3 раза. 
   Резиночки для брекетов изготовлены из высококачественного хирургического латекса. 
   Эти резинки на зубы являются неотъемлемой частью ортодонтического лечения и играют 
   важную роль в процессе коррекции исправления прикуса и выправления зубов. 
   Одной из основных функций ортодонтических тяг для брекетов является создание тяги и давления на зубы, 
   направляя их в нужное положение. Резинки для брекитов доступны в широком ассортименте цветов 
   и размеров, что позволяет каждому пациенту выбрать подходящий вариант для себя. 
   Яркие цветные резинки на брекеты помогают сделать процесс ношения брекетов более интересным
    и увлекательным, особенно для детей и подростков. В то же время, существуют 
    и бесцветные / прозрачные варианты резинок, которые практически незаметны на зубах и обеспечивают 
    более естественный внешний вид. Резинки для зубов поставляются в белых zip-lock пакетах 
    по 100 колец в каждом. Также каждый zip-lock пакет упаковывается в дополнительный запаянный пакет,
    чтобы обеспечить его безопасность и сохранность во время доставки до покупателей. 
    В продаже есть варианты с крючком / захватом для эластиков и без, на ваш выбор. 
    Эластики Ormco - одни из наиболее устойчивых с точки зрения потери силы со временем. Однако,
    как и все эластические продукты, они постепенно "устают". Для поддержания силового воздействия
    на более или менее постоянном уровне необходимо заменять эластики 3-4 раза в сутки.

Выводи ТОЛЬКО JSON-массив, ничего больше:
[
  {{"description": "...", "relevance_label": "label", "score": 0.XX}},
  ...
]
"""

In [103]:
def get_relevance_instructions(scores: tuple[float, float, float]) -> str:
    """
    Возвращает строку с инструкциями по уровням релевантности.
    
    """
    if len(scores) != len(RELEVANCE_LEVELS):
        raise ValueError(f'Ожидается кортеж ровно из {len(RELEVANCE_LEVELS)} значений score')
    
    defs = [
        f"совершенно нерелевантно (score: {scores[0]:.2f}): Товар из другой тематики, без какой-либо связи с постом (например, если пост про AI — товар про кухню).",
        f"средне релевантно (score: {scores[1]:.2f}): Частичная связь, но не полная. Здесь в части релевантности ориентируйся на то, какой score указан в сскобках (score: 0.XX)!",
        f"очень релевантно (score: {scores[2]:.2f}): Прямая связь с основной идеей поста (например, товар, который решает проблему из поста или связан с примером).",
    ]
    return "\n".join(f"• {defs[i]}" for i in range(len(RELEVANCE_LEVELS)))

In [104]:
def get_few_shot_examples() -> str:
    return """\
- Для нерелевантного: Пост про AI — описание "Кофеварка эспрессо De'Longhi, 15 бар, автоматический капучинатор, цена 12 000 руб." (никакой связи).
- Для средне релевантного: Пост про AI — описание "Курс по Python для начинающих, 50 уроков, цена 2 000 руб." (связь с программированием, косвенно к AI).
- Для очень релевантного: Пост про AI — описание "Подписка на ChatGPT Plus, интеграция нескольких моделей в чат, цена 1 500 руб/мес." (прямая связь с функцией поста).\
"""

In [191]:
client = OpenAI(base_url=API_BASE, api_key=API_KEY)

# Загрузка постов
# Если у вас уже есть список — подставьте его вместо чтения файла
# with open(INPUT_FILE, encoding="utf-8") as f:
#     posts: List[Dict[str, Any]] = json.load(f)

posts = [
# {"channel": "Alisa_Lazarenko", "text": "Девочки ❤️Записала для вас немножко полезностей ☺️✅ Как я рисую стрелки (в стрелках главное тренировки💪🏻 с первого раза мало у кого получается))✅Как правильно прокрашивать реснички✅Как отрастить длинные ресницы😜✅Показала один из любимых тонаков дорогих🙈✅И любимую маску для губСкоро я буду проводить урок макияжа для себя оффлайн 🙌🏻 2 часа полезной информации , секретов и нюансов красивого макияжа на каждый день🥰", "link": "https://t.me/Alisa_Lazarenko/146", "id": 146, "date": "2022-03-31T08:52:47+00:00", "views": "26.7K"},
# {"channel": "copyme", "text": "Прочитала в немецком Elle, что на столе у Миуччи Прады лежит маленькая книжица японского художника Сандзо Вада — «Словарь цветовых комбинаций», составленный еще в 1930-х годах. Это такая энциклопедия из трех сотен удивительных сочетаний оттенков, который Вада использовал для узоров кимоно. Читатели подсказывают, что вот по этой ссылке можно подбирать предложенные художником комбинации онлайн.", "link": "https://t.me/copyme/4122", "id": 4122, "date": "2023-04-05T08:29:12+00:00", "views": "71.2K"},
# {"channel": "cybersrach", "text": "Офлейнера Soniqs кикнули из команды из-за названия статуиВ матче против BOOM на англоязычной трансляции TI обсервер кликнул на статую Leslao, которая называется «Run nigga». После этого игрок попытался извиниться в твиттере:«Только что увидели сообщение на моей статуе. Оно явно неуместно и может задеть, прошу прощения за то, что я вообще написал его. Я небрежно написал это много лет назад, а потом забыл. Глубоко сожалею, что не проверил и не удалил это раньше»Но, извинения не помогли. Сегодня его кикнули из Soniqs 😨", "link": "https://t.me/cybersrach/2481", "id": 2481, "date": "2022-10-19T06:56:37+00:00", "views": "21.3K"},
# {"channel": "cybersrach", "text": "Ночью Вышел патч 7.35d. В патче два основных изменения: 🔹Теперь юзеры Dota Plus видят информацию о найденной игре и могут отклонить ее, не получив за это наказание🔹Теперь каждый игрок банит 4 персонажей ДО поиска игры. (1 из которых будет запрещен перед началом)Также затронули метовых героев и предметы: ослабили Black King Bar, Divine Rapier и Revenant’s Brooch. Из героев: Dragon Knight, Terrorblade, Mirana и других", "link": "https://t.me/cybersrach/6815", "id": 6815, "date": "2024-03-22T06:44:07+00:00", "views": "21.0K"},
# {"channel": "cybersrach", "text": "ПАТЧ – ЗАВТРАВ 1 акт «Павшей короны» добавили недостающую свечу, с помощью которой можно увидеть описание нового места и даже часть изменений нового патча:🔹 Базовая броня у ___ уменьшена на 0,5.🔹 Каждый раз, когда выходит солнце, ___ раскрывает всю карту на 5 секунд.🔹 Disruptor: Kinetic Field заменен на Kinetic ___🔹 Снаряды вражеской атаки в радиусе 500 от ___ замедляются.🔹 ___ может путешествовать по дуге🔹 ___ может использовать предметы в своем рюкзаке, как если бы они были в их инвентаре🔹 Silencer не может быть ___🔹 ___ не имеет уровней прокачки на 1 уровне🔹 Lifestealer навсегда получает ___ при убийстве крипа🔹 ___ начинает игру с дополнительными 250 золота🔹 Враги внутри ___ не могут видеть союзниковМнения?", "link": "https://t.me/cybersrach/7278", "id": 7278, "date": "2024-05-22T05:14:07+00:00", "views": "18.7K"},
# {"channel": "cybersrach", "text": "BetBoom распустила состав по Доте 😰«Сегодня мы прощаемся и говорим спасибо составу BetBoom Team. SoNNeikO, Daxak, Larl, Noticed и RodjER покидают нашу команду, но остаются в сердечках всех наших фанатов Желаем парням добиться ещё больших успехов и верим, что они ещё порубятся за титулы на тир-1 сцене   И может быть — против уже другой BetBoom Team   Спасибо, что были с нами!» – написал клуб", "link": "https://t.me/cybersrach/2642", "id": 2642, "date": "2022-11-03T18:44:54+00:00", "views": "17.4K"},
# {"channel": "cybersrach", "text": "Общий сбор! БП вышел 🤩«Откройте для себя новый боевой пропуск, поделённый на две части. Первая из них посвящена чествованию The International, с завершением которого и начнётся вторая. В ней мы отметим завершение турнирного сезона в морозной вариации классического «Восстания Тьмы». Следуя ежегодной традиции, пропуск наполнен первоклассными сокровищницами и функциями — как новыми, так и старыми, — которые точно наполнят вашу игру новыми украшениями и весельем» – Valve", "link": "https://t.me/cybersrach/2106", "id": 2106, "date": "2022-09-01T22:14:43+00:00", "views": "16.3K"},
# {"channel": "nachinay_vospitanie", "text": "Ты лежишь пока в своей кроватке И воркуешь беззаботно-сладко, Ты смеешься маме, погремушкам, Маленький сынок мой, лопотушка. Ты пока лишь с маминых ладоней Видишь мир, глазастик изумленный, Разноцветный, яркий и прекрасный, Каждый день все новый, звонкий, ясный. Но придет тот день и ты на ножки Встанешь неуверенно немножко, И пойдешь из комнаты в другую, А потом за дверь, во двор, ликуя. С каждым шагом все прочнее ноги, С каждым днем заманчивей дороги, С каждым годом все теснее дома, Неизбежно это и знакомо. А пока… и я смотрю на кроху, Нам для счастья нужно так немного, Молоко, игрушки, папа с мамой, Не спеши расти, мой самый-самый!", "link": "https://t.me/nachinay_vospitanie/482", "id": 482, "date": "2023-11-25T10:55:53+00:00", "views": "22.4K"},
# {"channel": "nachinay_vospitanie", "text": "Отложите в сторону дела, И найдите несколько минут, Чтобы детям подарить тепла, А дела, конечно, подождут… Обнимите сына или дочь, Расскажите о своей любви, Время метеором летит прочь, Совсем скоро вырастут они. Не ругайте вы по пустякам Тех, что дарят вам свой яркий свет. Дети же всем сердцем верят вам. Посидите с ними тет-а-тет, Загляните искренне в глаза, И прижмите крепко их к груди, Подберите добрые слова, Расскажите о своей любви! Не стесняйтесь чувства проявлять, Не держитесь вы от них в тени, Детям очень-очень важно знать, Что они на свете не одни. И что рядом с ними есть всегда Твердая опора их семьи, Отложите в сторону дела, И побудьте с вашими детьми!", "link": "https://t.me/nachinay_vospitanie/577", "id": 577, "date": "2024-03-06T12:23:11+00:00", "views": "21.4K"},
# {"channel": "nachinay_vospitanie", "text": "Cыну...  Ты вырастешь, и я не буду знать, С кем ты проводишь дни и даже ночи, Но я все так же буду называть Тебя как в детстве - милый мой сыночек.  Ты вырастешь... Ты будешь принимать Судьбы удары, думаю, достойно. И зубы стискивать и кулаки сжимать, А я уже не поцелую там, где больно.  И если с папой мы не справимся уже. Там кран потек или другие беды, Твой голос в трубке твердо скажет мне: \"Мамуль, не трогай ничего - сейчас приеду\".  Ну, а пока - пижамка в облаках, И на ночь про слона и Айболита. Твое сердечко в маминых руках, Душа еще ранима и открыта.  Я очень постараюсь, мой родной, Дать, что смогу, и отвести ненастья: Чтоб детство ты с улыбкой вспоминал, Чтоб знал, каким бывает счастье...", "link": "https://t.me/nachinay_vospitanie/599", "id": 599, "date": "2024-03-31T07:34:55+00:00", "views": "19.8K"},
# {"channel": "nachinay_vospitanie", "text": "Ты лежишь пока в своей кроватке И воркуешь беззаботно-сладко, Ты смеешься маме, погремушкам, Маленький сынок мой, лопотушка. Ты пока лишь с маминых ладоней Видишь мир, глазастик изумленный, Разноцветный, яркий и прекрасный, Каждый день все новый, звонкий, ясный. Но придет тот день и ты на ножки Встанешь неуверенно немножко, И пойдешь из комнаты в другую, А потом за дверь, во двор, ликуя. С каждым шагом все прочнее ноги, С каждым днем заманчивей дороги, С каждым годом все теснее дома, Неизбежно это и знакомо. А пока… и я смотрю на кроху, Нам для счастья нужно так немного, Молоко, игрушки, папа с мамой, Не спеши расти, мой самый-самый!", "link": "https://t.me/nachinay_vospitanie/527", "id": 527, "date": "2024-01-15T12:16:59+00:00", "views": "17.7K"},
# {"channel": "nachinay_vospitanie", "text": "КАКИЕ ДЕЛА МОЖНО ПОРУЧИТЬ ДЕТЯМ РАЗНОГО ВОЗРАСТА 📌Кapточки-шпаргалки в помощь для родителей📌Многие родители считают, что работа по дому будет отнимать у детей беззаботное детство, которое даётся только раз. У психологов иная точка зрения: когда дети помогают взрослым, они чувствуют себя полноправными членами семьи, учатся быть самостоятельными и не бояться ответственности.", "link": "https://t.me/nachinay_vospitanie/29", "id": 29, "date": "2022-08-25T04:02:44+00:00", "views": "12.8K"},
# {"channel": "Alisa_Lazarenko", "text": "Сегодня читала в инсте историю моей талантливой коллеги визажиста из Харькова … Ей кто то собрал ее кейс и передал, но доехало до неё практически ничего😭😭😭Все самое важное и ценное , кто-то в цепочке передающих спиздил 🤬Как вообще такое возможно? Это днище. Обычный человек не сможет всем этим воспользоваться…нет слов.", "link": "https://t.me/Alisa_Lazarenko/75", "id": 75, "date": "2022-03-16T21:09:05+00:00", "views": "35.0K"},
# {"channel": "Alisa_Lazarenko", "text": "Девочки! Нужна помощь🙏🏻 Набралась храбрости попросить 🙏🏻 помощь заключается в том,что помочь найти, именно найти! (А я куплю) две белые кровати с матрасами односпальные (б/у) в нормальном состоянии. Типа икеа или икеа🙏🏻 сама ищу,но пока безрезультатно (( Может кто то видел  или продаёт лично.🙏🏻 (Москва)", "link": "https://t.me/Alisa_Lazarenko/47", "id": 47, "date": "2022-03-12T16:04:28+00:00", "views": "31.4K"},
# {"channel": "Alisa_Lazarenko", "text": "Кровати приехали так быстро😳💪🏻И это все благодаря ВАМ!!! (Они новые 5000 рублей за кровать) уже собирают🙏🏻сейчас родители привезу детей на смотрины☺️Сегодня одна добрая девушка Юля привезла детский ортопедический стул и много всего для кухни🙏🏻Я думаю, ещё пару дней и можно будет жить 🤞🏻Спасибо вам девочки за вашу поддержку,помощь,денежные переводы🙈🙏🏻❤️", "link": "https://t.me/Alisa_Lazarenko/53", "id": 53, "date": "2022-03-13T14:42:42+00:00", "views": "30.8K"},
# {"channel": "Alisa_Lazarenko", "text": "Два года назад у меня был жуткий стресс, я бы сказала потрясение 🙈Думала конец света, но как видите я жива😆 Так вот, я в тот период ходила по 2-3 тренировки в день каждый день 🤪 И только этим и спасалась… сегодня мама меня вытащила на тренировку💪🏻какой же это кайф 😻 девочки, как часто вы ходите на треши и ходите ли вообще?👀", "link": "https://t.me/Alisa_Lazarenko/105", "id": 105, "date": "2022-03-22T09:19:06+00:00", "views": "30.3K"},
# {"channel": "autohaykcatalog", "text": "Hyundai Elantra Sel 2018❗️В наличии❗️VIN - 5NPD84LF1KH448223✔️ Объем двигателя - 2.0 L✔️ Мощность - 150 л.с.✔️ Бензиновый двигатель✔️ Расход на 100 км. - 7.0 л.✔️ Пробег - 111200 км.✔️ Передний привод✔️ Коробка - автомат (АКПП)✔️ Электроусилитель руля✔️ Дисковые тормоза✔️ Галогенные фары✔️ Датчик света✔️ Датчик дождя✔️ Электропривод боковых зеркал✔️ Электроподогрев зеркал✔️ Автозатемнение зеркал✔️ Обогрев заднего стекла✔️ Мультируль✔️ Бесключевой доступ✔️ Электроподогрев передних сидений✔️ Электрические стеклоподъемники передние✔️ Электрические стеклоподъемники задние✔️ Системаконтроля»слепых зон»✔️ Адаптивные фары✔️ Система контроля полосы✔️ Адаптивный круиз-контроль✔️ Климат-контроль✔️ КондиционерЦена - 17.500$", "link": "https://t.me/autohaykcatalog/7298", "id": 7298, "date": "2022-07-23T11:18:45+00:00", "views": "9.4K"},
{"channel": "autohaykcatalog", "text": "Kia Optima Lx 2,4 2020 годЦелая Optima 🔥Старт аукциона - через 3 дня#лота - 48710902VIN - 5XXGT4L34LG435948✔️ Объём двигателя - 2,4L✔️ Передний привод✔️ Пробег - 57.500 км✔️ Камера заднего обзора✔️ Мультируль✔️ Автозатемнение зеркалЦена - 20.000$Примерная цена под ключ на учёт России (с учётом ремонта)Успей заказать", "link": "https://t.me/autohaykcatalog/4937", "id": 4937, "date": "2022-06-19T14:22:37+00:00", "views": "9.1K"},
# {"channel": "autohaykcatalog", "text": "https://bid.cars/ru/lot/1-66859271/2016-Mitsubishi-Outlander-JA4AP3AU7GZ0658352016 год17.000$Mitsubishi Outlander Sport Es 2016❗️В наличии❗️Цена в Армении❗️VIN - JA4AP3AU7GZ065835✔️ Объем двигателя - 2.0 L✔️ Мощность - 150 л.с.✔️ Бензиновый двигатель✔️ Расход на 100 км. - 6.3 л.✔️ Пробег - 108100 км.✔️ Передний привод✔️ Коробка - автомат (вариатор)✔️ Система старт-стоп✔️ Электроусилитель руля✔️ Дисковые тормоза✔️ Электрический ручник✔️ Галогенные фары✔️ Электроподогрев зеркал✔️ Автозатемнение зеркал✔️ Обогрев заднего стекла✔️ Электроподогрев лобового стекла✔️ Мультируль✔️ Электроподогрев передних сидений✔️ Электрические стеклоподъемники передние✔️ Электрические стеклоподъемники задние✔️ Круиз-контроль✔️ Раздельный климат-контроль✔️ Камера заднего ходаЦена - 17.000$", "link": "https://t.me/autohaykcatalog/7314", "id": 7314, "date": "2022-07-24T08:35:16+00:00", "views": "8.6K"},
# {"channel": "autohaykcatalog", "text": "время знакомства ⚜🕊Меня зовут Лиза ✔Я сотрудник Ставропольского офиса , компании AUTOHAYK @auto_haykЯ честно и безответно влюблена в свою работу🚘 и стараюсь всегда вложить каждый раз частичку своей души.Обратная связь для меня необходима, все свое внимание стараюсь отдавать работе🚘Я рада каждому из вас , кто находится на нашем аккаунте ⚜Я буду рада ответить на все ваши вопросы , и с радостью просчитаю любой интересующий вас автомобиль📍проспект Кулакова 65📲+7-989-120-00-26", "link": "https://t.me/autohaykcatalog/7374", "id": 7374, "date": "2022-07-25T11:33:14+00:00", "views": "8.5K"},
# {"channel": "autohaykcatalog", "text": "Ford Mustang Ecoboost Premium 2015 ❗️В наличии❗️29.000$ ✔️ Объем двигателя - 2.3 L✔️ Мощность - 305 л.с.✔️ Бензиновый двигатель✔️ Задний привод✔️ Расход на 100 км. - 9.8 л.✔️ Пробег - 190300 км.✔️ Коробка - автомат (АКПП)✔️ Электроусилитель руля✔️ Дисковые тормоза✔️ Ксеноновые фары✔️ Датчик света✔️ Датчик дождя✔️ Электропривод боковых зеркал✔️ Электроподогрев зеркал✔️ Обогрев заднего стекла✔️ Кожаный салон✔️ Мультируль✔️ Электропривод передних сидений✔️ Электроподогрев передних сидений✔️ Вентиляция передних сидений✔️ Бесключевой доступ✔️ Электрические стеклоподъемники передние✔️ Круиз-контроль✔️ Раздельный климат-контроль✔️ Кондиционер✔️ Камера заднего хода", "link": "https://t.me/autohaykcatalog/7680", "id": 7680, "date": "2022-07-31T07:47:54+00:00", "views": "8.3K"},
# {"channel": "arbitraj_delay", "text": "Важно! Рекламодателям!Павел Дуров анонсировал Stories в Телеграм уже в начале июляКак вы понимаете, истории в личных профилях с возможностью репоста из групп увеличат охваты просмотров и актив на страницах и это отличная возможность для вашего бизнеса развивать его в два раза активнееОбращайтесь к нашему менеджеру (в шапке профиля) и он подберёт для вас лучший вариант размещения (у нас больше 3 тысяч каналов с суммарной аудиторией в 150 миллионов человек)P.S. Успейте до глобального повышения цен на фоне увеличения актива =)", "link": "https://t.me/arbitraj_delay/329", "id": 329, "date": "2023-06-27T10:19:33+00:00", "views": "23.0K"},
# {"channel": "arbitraj_delay", "text": "Нейро-художник с помощью Midjourney пофантазировал, как могли бы выглядеть бонусы подписки Плюс, если бы они были буквально связаны. На артах вы можете увидеть Яндекс Музыку, Кинопоиск, баллы Плюса, Букмейт и Плюс Сити.Только посмотрите на эту милейшую колонкуНейро-художник с помощью Midjourney пофантазировал, как могли бы выглядеть бонусы подписки Плюс, если бы они были буквально связаны. На артах вы можете увидеть Яндекс Музыку, Кинопоиск, баллы Плюса, Букмейт и Плюс Сити.", "link": "https://t.me/arbitraj_delay/200", "id": 200, "date": "2023-02-17T16:12:43+00:00", "views": "22.0K"},
# {"channel": "arbitraj_delay", "text": "Парень из США собрал свой ПК из комплектующих, найденных на мусорках возле магазинов техники – на этого к него ушло полгода.Внутри: видеокарта EVGA GTX 570, Intel Core I7-3770 Non-K, 16 ГБ Corsair Vengeance , блок питания на 750-ватт от Corsair, корпус iBUYPOWER Snowblind Element, материнка Dell OptiPlex 9010. Рабочий БП и ОЗУ найти не удалось - он их докупил за 140$.", "link": "https://t.me/arbitraj_delay/255", "id": 255, "date": "2023-04-15T18:53:18+00:00", "views": "21.7K"},
# {"channel": "singularityp0int", "text": "🧬 Чат-бот Илона Маска #Grōk стал доступен для платных подписчиков 💬 в Европе Grok отличается от других ассистентов тем, что предоставляет информацию в реальном времени, используя Twitter/X в качестве базы знаний, стремится быть максимально полезным, остроумным, и не боится острых вопросов или спорных тем. Grōk всё ещё находится в разработке и может не всегда понимать, что вы от него хотите. Например, он соглашается всегда писать на русском, вместо английского, но спустя пару сообщений забывает об этом.🧩 #AINews", "link": "https://t.me/singularityp0int/3860", "id": 3860, "date": "2024-05-19T10:15:04+00:00", "views": "10.5K"},
# {"channel": "singularityp0int", "text": "🧬 Очередной свеженький #КаталогНейросетей: более 4000 актуальных нейросетей в одном месте!gpte.ai — хранит в себе все топовые нейросети из разных сфер: программирование, дизайн, бизнес, текста и т.д. Есть как платные, так и бесплатные, найти что-то под свои нужды вы точно сможете. Также есть удобный поисковик.🧩 #КаталогНейросетей", "link": "https://t.me/singularityp0int/2974", "id": 2974, "date": "2024-02-11T12:00:39+00:00", "views": "10.3K"},
# {"channel": "singularityp0int", "text": "🧬 Олимпиаду в Париже будет комментировать нейросетьПо сообщениям СМИ в роли комментатора на грядущих Играх выступит цифровой ИИ-клон 79-летнего спортивного диктора и телеведущего Эла Майклза.Нейронке собираются скормить около 7 миллионов фраз, которые она будет использовать во время спортивных выступлений. Сам Майклз вначале был настроен к этому скептически, но затем послушал демонстрацию технологии и сказал: «Я в деле!».🧩 #AINews", "link": "https://t.me/singularityp0int/4368", "id": 4368, "date": "2024-07-07T13:20:02+00:00", "views": "10.2K"},
# {"channel": "singularityp0int", "text": "🧬 #OpenAI официально запустили функцию вызова разных GPT в одном чате.Набрав \"@\", а затем название GPT, можно вызвать в один чат разные GPT. Это позволяет создать более персонализированного универсального ассистента в одном окне, который сможет связывать ответы и навыки каждого GPT друг с другом. На примере представлена словесная дуэль чат-ботов с личностями Трампа и Байдена. По тексту видно, что GPT ссылаются на слова друг друга.🧩 #AINews", "link": "https://t.me/singularityp0int/2886", "id": 2886, "date": "2024-02-02T09:44:01+00:00", "views": "10.0K"},
# {"channel": "singularityp0int", "text": "🧬 Google Cloud и Hugging Face стали стратегическими партнерами.Популярная платформа для машинного обучения получит доступ к вычислительным ресурсам Google Cloud и платформе для тренировки и развертывания моделей Vertex AI. Сообщается, что сотрудничество с Google будет в области с открытым научным исследованием, с открытым исходным кодом, облачных технологий и оборудования, чтобы компании могли создавать свои собственные ИИ, используя новейшие открытые модели от Hugging Face и новейшие облачные и аппаратные функций от Google Cloud.Обе компании обещают, что новое партнерство позволит сделать ИИ с открытым исходным кодом еще доступнее.🧩 #AINews", "link": "https://t.me/singularityp0int/2868", "id": 2868, "date": "2024-01-31T07:37:02+00:00", "views": "9.9K"},
# {"channel": "emperia_film", "text": "УЖЕ СЛЫШАЛИ ОБ ЭТОМ❓Оригинальная техника Apple прямиком из Европы!  - ПО ЦЕНЕ НИЖЕ РЫНКА ❗️ТАКОГО ВЫ ТОЧНО НЕ НАЙДЁТЕ НИ В ОДНОМ МАГАЗИНЕ И МАРКЕТПЛЕЙСЕ! -Айфоны 📱 -Макбуки 👩‍💻 -Часы ⏰ -Планшеты 📱 Эти ребята отвечают за качество и дают заводскую гарантию!*а ещё можно купить оптом🤫Скорее переходи в канал, пока он открыт 💪https://t.me/Smarton_deviceA1032", "link": "https://t.me/emperia_film/4667", "id": 4667, "date": "2024-02-26T17:00:10+00:00", "views": "7.4K"},
# {"channel": "emperia_film", "text": "Фильм №22 - Келин особого назначения (2022)#комедия Сотрудница спецслужб Асель по ошибке принимает за бандита обычного парня Ерлана, который увозит её в деревню в качестве будущей невесты. Теперь, чтобы вывести подозреваемого и его семью на чистую воду, девушка должна вести домашнее хозяйство, о котором совсем ничего не знает.🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/244", "id": 244, "date": "2023-02-22T05:30:09+00:00", "views": "7.0K"},
# {"channel": "emperia_film", "text": "Фильм №186 -  Один вдох (2020)#драма В сорок лет Марина Гордеева неожиданно открывает для себя фридайвинг – экстремальное подводное плавание. Погружаясь на глубину, она бросает вызов своему страху и пределу человеческих возможностей. Виктория Исакова, Владимир Яглыч, Максим Суханов, Артем Ткаченко и Стася Милославская в перехватывающей дыхание спортивной драме Елены Хазановой «Один вдох». Прототипом главной героини стала «королева фридайвинга» Наталья...🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/2319", "id": 2319, "date": "2023-07-09T20:59:34+00:00", "views": "6.8K"},
# {"channel": "emperia_film", "text": "Фильм №17 - Убийство онлайн(2021)#триллеры Действие остросюжетного триллера разворачивается в 2022 году. Пандемия COVID-19 всё ещё бушует в мире, и в США находится в затяжном карантине. На этом фоне семеро друзей, живущие в разных уголках страны, решили всё-таки провести вечеринку, но в режиме онлайн, чтобы поздравить одного из героев Эвана с днём рождения.🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/201", "id": 201, "date": "2023-02-19T18:31:42+00:00", "views": "6.6K"},
# {"channel": "emperia_film", "text": "Фильм №40 -  На склоне (2020)#драма Благополучная семейная пара Пит и Билли приезжают вместе с детьми на горнолыжный курорт. Неожиданно они оказываются в экстремальной ситуации, пережив сход лавины, которая погребает под собой их счастливый брак. Оказывается, при наступлении опасности самые близкие люди могут вести себя совсем не так, как мы от них ожидаем…🍿СМОТРЕТЬ", "link": "https://t.me/emperia_film/367", "id": 367, "date": "2023-02-28T07:31:09+00:00", "views": "6.5K"},
# {"channel": "copyme", "text": "Time опубликовали первый отрывок из биографии Анны Винтур. Он как раз рассказывает об организации Met Gala. Есть смешной момент, когда Анна просила помощников наконец-то усадить Ким Кардашьян за стол, но хозяйка бала не учла, что в этом платье Mugler Кардашьян просто физически не могла сесть. Зачем-то припомнили Харви Вайнштейна. Говорят, он лихорадочно добивался расположения Анны еще с середины 1990-х и в итоге добился: по крайней мере на Met Gala Анна распоряжалась, чтобы его пропускали сразу, вытаскивая из очереди на красную дорожку.", "link": "https://t.me/copyme/1569", "id": 1569, "date": "2022-04-26T15:11:23+00:00", "views": "109.3K"},
# {"channel": "copyme", "text": "Умеет же Джонатан Андерсон выбирать героев :) Это женщина в новой рекламе Loewe — художница по костюмам Сэнди Пауэлл. Она отвечала за образы в «Орландо», «Волке с Уолл-Стрит», «Фаворитке» — всего у Сэнди 15 номинаций на «Оскар» и три победы. Пару лет назад она посещала киноцеремонии в светлом костюме из простого сукна и с черными фломастерами в карманах — и просила знаменитостей расписаться на пиджаке и брюках. В итоге костюм вместил более 200 подписей, включая Ди Каприо, Скарлетт Йоханссон, Роберта де Ниро, Скорсезе и Тильду. Дальше Пауэлл отправила костюм на аукцион — а на вырученные деньги спасла резиденцию своего друга, режиссера Дерека Джармена, которую хотели снести. Сейчас костюм Сэнди Пауэлл хранится в Музее Виктория и Альберта.", "link": "https://t.me/copyme/2635", "id": 2635, "date": "2022-11-14T11:23:42+00:00", "views": "76.8K"},
# {"channel": "copyme", "text": "«Я обычная девчонка из Техаса, и на мне платье Schiaparelli, которое сделал мой брат Дэниел!». В тиктоке взлетело трогательное видео со свадьбы некой Лиз Фокс Розберри — как оказалось, сестры креативного директора Schiaparelli Дэниела Розберри. На бирке платья дизайнер собственноручно вышил слово Зибо — детское прозвище сестренки.", "link": "https://t.me/copyme/1637", "id": 1637, "date": "2022-05-12T21:05:28+00:00", "views": "66.2K"},
# {"channel": "copyme", "text": "Здесь даже без шуток — дизайнер Сабато де Сарно придумал эти туфли Gucci, когда недавно в Будапеште встретил двух венгерских овчарок (комондор). Сабато в жизни таких не видел — в его родном Неаполе, говорит, таких нет, у них там либо низкорослые борзые в ошейниках со стразами, либо свирепые мастифы, которые охраняют виллы с видом на Везувий. «Первая мысль была — сделать туфли в честь этих венгерских собак. Смешные свисающие туфли».", "link": "https://t.me/copyme/7791", "id": 7791, "date": "2024-03-25T11:21:18+00:00", "views": "54.8K"}
]

print(f"Загружено постов: {len(posts):,}")
print("Пример первого поста:", posts[0]["text"][:120], "...")

Загружено постов: 1
Пример первого поста: Kia Optima Lx 2,4 2020 годЦелая Optima 🔥Старт аукциона - через 3 дня#лота - 48710902VIN - 5XXGT4L34LG435948✔️ Объём двиг ...


In [240]:
def generate_product_descriptions(post_text: str): # type: ignore
    """Возвращает список словарей с description, relevance_label, score"""
    
    prompt = PROMPT_TEMPLATE.format(
        post_text = post_text.strip(),
        num_desc  = NUM_DESCRIPTIONS,
        relevance_instructions = get_relevance_instructions(SCORES),
        few_shot_examples = get_few_shot_examples(),
        min_description_symbols = MIN_DESCRIPTION_SYMBOLS,
        max_description_symbols = MAX_DESCRIPTION_SYMBOLS,
        low_score=SCORES[0],
        mid_score=SCORES[1],
        high_score=SCORES[2]
    )

    print(prompt)
    
    try:
        resp = client.chat.completions.create(
            model       = MODEL_NAME,
            messages    = [{"role": "user", "content": prompt}],
            temperature = TEMPERATURE,
            max_tokens  = MAX_TOKENS,
            top_p       = TOP_P,
        )
        
        text = resp.choices[0].message.content.strip()
        
        # Убираем возможные ```json ... ``` обёртки
        if text.startswith("```json"):
            text = text.split("```json", 1)[1].split("```", 1)[0].strip()
        elif text.startswith("```"):
            text = text.split("```", 2)[1].strip()
        
        data = json.loads(text)
        
        if not isinstance(data, list) or len(data) != NUM_DESCRIPTIONS:
            print("Неверный формат ответа →", text[:200])
            return []
            
        return data
        
    except Exception as e:
        print("Ошибка при генерации:", str(e))
        return []

In [202]:
output_pairs: List[Dict[str, Any]] = []

for i, post in enumerate(tqdm(posts, desc="Генерация описаний")):
    descs = generate_product_descriptions(post["text"])

    print(descs)
    
    if len(descs) != NUM_DESCRIPTIONS:
        print(f"Пост {post.get('id', '???')} → получено {len(descs)} описаний вместо {NUM_DESCRIPTIONS}")
    
    for d in descs:
        pair = {
            "description":      d.get("description", "").strip(),
            "post_text":        post["text"],
            "score":            float(d.get("score", 0.0)),
            "relevance_label":  d.get("relevance_label", ""),
            
            # Оригинальные поля поста
            "channel":          post.get("channel", ""),
            "post_id":          post.get("id", None),
            "link":             post.get("link", ""),
            "date":             post.get("date", ""),
            "views":            post.get("views", ""),
        }
        output_pairs.append(pair)
    
    # Промежуточное сохранение
    # if (i + 1) % SAVE_EVERY == 0:
    #     with open(f"intermediate_{i+1}.json", "w", encoding="utf-8") as f:
    #         json.dump(output_pairs, f, ensure_ascii=False, indent=2)
    #     print(f"  → промежуточный файл сохранён ({len(output_pairs)} пар)")

Генерация описаний:   0%|          | 0/1 [00:00<?, ?it/s]

Ты — генератор синтетических описаний товаров строго по заданной степени релевантности.  
Твоя задача — создать описания, которые **точно соответствуют указанному score**, а не просто "какие-то похожие".

На основе поста из Telegram:
«Kia Optima Lx 2,4 2020 годЦелая Optima 🔥Старт аукциона - через 3 дня#лота - 48710902VIN - 5XXGT4L34LG435948✔️ Объём двигателя - 2,4L✔️ Передний привод✔️ Пробег - 57.500 км✔️ Камера заднего обзора✔️ Мультируль✔️ Автозатемнение зеркалЦена - 20.000$Примерная цена под ключ на учёт России (с учётом ремонта)Успей заказать»

Сгенерируй РОВНО 3 описаний товаров. Каждое описание должно иметь **строго заданную** степень релевантности:

• совершенно нерелевантно (score: 0.10): Товар из другой тематики, без какой-либо связи с постом (например, если пост про AI — товар про кухню).
• средне релевантно (score: 0.56): Частичная связь, но не полная. Здесь в части релевантности ориентируйся на то, какой score указан в сскобках (score: 0.XX)!
• очень релевантно (score: 0.93

Генерация описаний:   0%|          | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [178]:
_GIGA_CLIENT: GigaChat | None = None
_GIGA_TOKEN_EXPIRES_AT: float = 0.0
_GIGA_CREDENTIALS  =os.getenv("GIGA_CHAT_API_KEY")


In [111]:
def _get_giga_client() -> GigaChat:
    global _GIGA_CLIENT, _GIGA_TOKEN_EXPIRES_AT

    now = time.time()

    if _GIGA_CLIENT is None or now >= _GIGA_TOKEN_EXPIRES_AT:
        giga = GigaChat(
            credentials=_GIGA_CREDENTIALS,
            verify_ssl_certs=False,
        )
        response = giga.get_token()

        _GIGA_CLIENT = GigaChat(
            access_token=response.access_token,
            verify_ssl_certs=False,
        )
        _GIGA_TOKEN_EXPIRES_AT = now + 30 * 60  # 30 минут

    return _GIGA_CLIENT


In [251]:
# def parse_gigachat_response(raw_response: str) -> List[Dict[str, Any]]: # type: ignore
#     """
#     Очищает и парсит ответ от GigaChat.
#     Возвращает список словарей или [] при ошибке.
#     """
#     try:
#         # 1. Берём текст ответа
#         text = raw_response
        
#         # 2. Убираем типичные обёртки
#         if text.startswith("```json"):
#             text = text.split("```json", 1)[1]
#         if text.startswith("```"):
#             text = text.split("```", 2)[-1] if "```" in text else text
#         if text.endswith("```"):
#             text = text.rsplit("```", 1)[0]
        
#         text = text.strip()
        
#         # 3. Убираем возможные префиксы/суффиксы (типа "Вот JSON:" или пустые строки)
#         lines = text.splitlines()
#         cleaned_lines = [line for line in lines if line.strip() and not line.strip().startswith(("Вот", "Ответ", "JSON", "```"))]
#         cleaned_text = "\n".join(cleaned_lines).strip()
        
#         # 4. Пытаемся распарсить
#         parsed = json.loads(cleaned_text)
        
#         if not isinstance(parsed, list):
#             raise ValueError(f"Ответ не является списком, а {type(parsed).__name__}")
        
#         # 5. Проверяем структуру (опционально, но полезно)
#         for item in parsed:
#             if not isinstance(item, dict) or "description" not in item:
#                 raise ValueError("Элемент списка не является словарем с ключом 'description'")
        
#         return parsed
    
#     except json.JSONDecodeError as je:
#         print("❌ JSONDecodeError при парсинге GigaChat")
#         print("Сырой текст:", repr(raw_response))  # первые 500 символов
#         print("Ошибка:", je)
#         return []
    
#     except Exception as e:
#         print("❌ Общая ошибка парсинга GigaChat")
#         print("Сырой текст:", repr(raw_response))
#         print("Ошибка:", e)
#         return []


def parse_gigachat_response(raw_response: str) -> List[Dict[str, Any]]:
    """
    Очистка и парсинг ответа от GigaChat.
    Чинит незакрытый массив, если нет закрывающей ].
    """
    try:
        text = raw_response.strip()

        # 1. Убираем ```json
        if text.startswith("```json"):
            text = text[7:].lstrip()
        elif text.startswith("```"):
            text = text[3:].lstrip()

        if text.endswith("```"):
            text = text[:-3].rstrip()

        # 2. Удаляем типичные префиксы
        common_prefixes = [
            "Вот JSON:", "Вот ответ:", "Ответ:", "JSON:",
            "Вот массив:", "Результат:", "Вот:",
        ]
        for prefix in common_prefixes:
            if text.lower().startswith(prefix.lower()):
                text = text[len(prefix):].lstrip()

        # 3. Убираем пустые строки
        lines = [line for line in text.splitlines() if line.strip()]
        text = "\n".join(lines).strip()

        # 4. 🔧 ПРОСТОЙ И НАДЁЖНЫЙ ФИКС
        # Если это массив, но нет закрывающей ]
        text = re.sub(r'}\s*$', '}', text)
        if text.startswith("[") and not text.endswith("]"):
            text += "]"

        # 5. Парсинг
        parsed = json.loads(text)
        if isinstance(parsed, list):
            return _validate_list(parsed)

        return []

    except Exception as e:
        print("❌ Ошибка парсинга GigaChat")
        print("Сырой текст:", repr(raw_response[:600]))
        print("После очистки:", repr(text))
        print("Ошибка:", e)
        return []


def _validate_list(items: list) -> list:
    for item in items:
        if not isinstance(item, dict) or "description" not in item:
            raise ValueError("Элемент не dict или нет ключа 'description'")
    return items

In [226]:
giga = _get_giga_client()

In [252]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

# Глобальный семафор (создаётся один раз вне функции)
_semaphore = asyncio.Semaphore(1)


async def generate_product_descriptions_gigachat(post_text: str) -> List[Dict[str, Any]]:
        
    async with _semaphore:  # ← здесь семафор, ограничивает параллелизм
        try:
            prompt = PROMPT_TEMPLATE.format(
                post_text=post_text.strip(),
                num_desc=NUM_DESCRIPTIONS,
                relevance_instructions=get_relevance_instructions(SCORES),
                few_shot_examples=get_few_shot_examples(),
                min_description_symbols=MIN_DESCRIPTION_SYMBOLS,
                max_description_symbols=MAX_DESCRIPTION_SYMBOLS,
                low_score=SCORES[0],
                mid_score=SCORES[1],
                high_score=SCORES[2]
            )
            
            # Асинхронный вызов chat
            response = await giga.achat(prompt)
            # await asyncio.sleep(random.uniform(0.8, 10))
            
            print('max raw response:', response.choices[0].message.content)

            response_descriptions = parse_gigachat_response(response.choices[0].message.content)
            print('desc:', response_descriptions)
            
            if not isinstance(response_descriptions, list):
                raise ValueError("Ответ не список")
            return response_descriptions
        except Exception as e:
            print("❌ Ошибка парсинга ответа GigaChat")
            print('ошибка:', e)
            return []

In [257]:
output_pairs: List[Dict[str, Any]] = []
for i, post in enumerate(tqdm(posts, desc="Генерация описаний")):
    descs = await generate_product_descriptions_gigachat(post["text"])
    for d in descs:
        pair = {
            "description":      d.get("description", "").strip(),
            "post_text":        post["text"],
            "score":            float(d.get("score", 0.0)),
            "relevance_label":  d.get("relevance_label", ""),
            "channel":          post.get("channel", ""),
            "post_id":          post.get("id", None),
            "link":             post.get("link", ""),
            "date":             post.get("date", ""),
            "views":            post.get("views", ""),
        }
        output_pairs.append(pair)

Генерация описаний: 100%|██████████| 1/1 [00:03<00:00,  3.15s/it]

max raw response: [
  {
    "description": "Устройство очистки воды в бассейне Robust AquaClean 5000 очищает воду бассейна от загрязнений, водорослей и бактерий, поддерживая её прозрачность и свежесть. С помощью ультрафиолетового излучения вода подвергается стерилизации, предотвращающей размножение микроорганизмов. Подходит для круглогодичной эксплуатации, оснащено удобным цифровым дисплеем для настройки режимов работы и системой оповещения о необходимости обслуживания фильтра. Эффективно удаляет загрязнения и предотвращает образование известкового налета благодаря автоматической обратной промывке фильтрующего элемента.", 
    "relevance_label": "совершенно нерелевантно", 
    "score": 0.10
  },
  {
    "description": "Персональный фитнес-трекер Xiaomi Mi Band 7 оснащен ярким AMOLED-дисплеем, сенсорами сердечного ритма и сна, шагомером, функцией подсчета калорий и GPS-навигацией. Благодаря компактному дизайну и водонепроницаемости устройство идеально подходит для активного образа жизни

In [258]:
pd.set_option("display.max_colwidth", None)
# Сохранение результата
# with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
#     json.dump(output_pairs, f, ensure_ascii=False, indent=2)

# print(f"\nГотово! Сохранено {len(output_pairs):,} пар в {OUTPUT_FILE}")

# Быстрый обзор
df = pd.DataFrame(output_pairs)
print("\nРаспределение score:")
print(df["score"].value_counts().sort_index())

print("\nПример 3 случайных записей:")
df[["description", "post_text", "score", "relevance_label"]]


Распределение score:
score
0.10    1
0.56    1
0.93    1
Name: count, dtype: int64

Пример 3 случайных записей:


,description,post_text,score,relevance_label
0,"Устройство очистки воды в бассейне Robust AquaClean 5000 очищает воду бассейна от загрязнений, водорослей и бактерий, поддерживая её прозрачность и свежесть. С помощью ультрафиолетового излучения вода подвергается стерилизации, предотвращающей размножение микроорганизмов. Подходит для круглогодичной эксплуатации, оснащено удобным цифровым дисплеем для настройки режимов работы и системой оповещения о необходимости обслуживания фильтра. Эффективно удаляет загрязнения и предотвращает образование известкового налета благодаря автоматической обратной промывке фильтрующего элемента.","Kia Optima Lx 2,4 2020 годЦелая Optima 🔥Старт аукциона - через 3 дня#лота - 48710902VIN - 5XXGT4L34LG435948✔️ Объём двигателя - 2,4L✔️ Передний привод✔️ Пробег - 57.500 км✔️ Камера заднего обзора✔️ Мультируль✔️ Автозатемнение зеркалЦена - 20.000$Примерная цена под ключ на учёт России (с учётом ремонта)Успей заказать",0.10,совершенно нерелевантно
1,"Персональный фитнес-трекер Xiaomi Mi Band 7 оснащен ярким AMOLED-дисплеем, сенсорами сердечного ритма и сна, шагомером, функцией подсчета калорий и GPS-навигацией. Благодаря компактному дизайну и водонепроницаемости устройство идеально подходит для активного образа жизни и занятий спортом. Поддерживает подключение к смартфонам на базе Android и iOS, синхронизация осуществляется через приложение Mi Fit. Универсальное решение для мониторинга физической активности и здоровья, удобное и практичное в повседневном использовании.","Kia Optima Lx 2,4 2020 годЦелая Optima 🔥Старт аукциона - через 3 дня#лота - 48710902VIN - 5XXGT4L34LG435948✔️ Объём двигателя - 2,4L✔️ Передний привод✔️ Пробег - 57.500 км✔️ Камера заднего обзора✔️ Мультируль✔️ Автозатемнение зеркалЦена - 20.000$Примерная цена под ключ на учёт России (с учётом ремонта)Успей заказать",0.56,средне релевантно
2,"Набор инструментов Kia Optima LX 2020 года выпуска включает передние и задние амортизаторы, стабилизаторы поперечной устойчивости, ступичные подшипники и рулевые тяги. Набор предназначен для профессионального ремонта автомобилей Kia, обеспечивает высокую надежность и долговечность деталей подвески. Использование оригинальных запчастей Kia гарантирует точное соответствие заводским параметрам и высокое качество сборки. Все детали проходят строгий контроль качества и сертифицированы производителем. Идеально подходит для тех, кто ценит надёжность и уверенность на дороге.","Kia Optima Lx 2,4 2020 годЦелая Optima 🔥Старт аукциона - через 3 дня#лота - 48710902VIN - 5XXGT4L34LG435948✔️ Объём двигателя - 2,4L✔️ Передний привод✔️ Пробег - 57.500 км✔️ Камера заднего обзора✔️ Мультируль✔️ Автозатемнение зеркалЦена - 20.000$Примерная цена под ключ на учёт России (с учётом ремонта)Успей заказать",0.93,очень релевантно
